In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.utils import class_weight
from keras import regularizers
import random
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
from tensorflow.keras import models
from google.colab import drive
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


drive.mount('/content/drive/')
out_path = '/content/drive/MyDrive/Barsellotti_Paolini'

def toRGB(images):
  tmp_images = []
  for image in images:
    reshaped_image = image.reshape((150,150,1))
    rgb_image = np.repeat(reshaped_image, 3, 2)
    rgb_image = rgb_image / 65535
    tmp_images.append(rgb_image)
  tmp_images = np.array(tmp_images)
  return tmp_images

model = tf.keras.models.load_model(os.path.join(out_path, "models/ensembling_stacking_bm"))
test_images_private = np.load(out_path+ '/private_test_tensor.npy')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
x_test_baseline = test_images_private[0::2] 
x_test_abno = test_images_private[1::2] # remove baseline if you don't use it

rgb_test_images = toRGB(x_test_abno)
test_images_tensor = x_test_abno / 65535
test_images_tensor = test_images_tensor.reshape((336, 150, 150, 1))

prediction = (model.predict((test_images_tensor, rgb_test_images)) > 0.5).astype("int32")
output_labels = prediction


print("Len prediction: ",len(prediction))
print("Shape prediction: ",np.array(prediction).shape)
print("Malignant classification: ",np.count_nonzero(output_labels))


assert len(output_labels) == 336 # must be of this size!

with open(out_path+'/Submission.csv','w') as mf:
  mf.writelines("\n".join(str(i[0]) for i in output_labels))

Len prediction:  336
Shape prediction:  (336, 1)
Malignant classification:  131
